In [3]:
### I/O and OS functions
from pprint import pprint
import os,sys,time

import json,csv


### Selenium
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from unidecode import unidecode ## chuyển tiếng việt thành không dấu.
#CURRENT_WORKING_DIRECTORY = os.path.dirname(os.path.abspath(__file__))
CURRENT_WORKING_DIRECTORY = r'd:\HCMUS\Data Engineer\project'
DATA_DIRECTORY = os.path.join(CURRENT_WORKING_DIRECTORY,'data')
    
    
print("working directory: " + CURRENT_WORKING_DIRECTORY)
print("data directory: " + DATA_DIRECTORY)
log_file = open("crawl_log.txt",'w',encoding='utf8')
def log_print(string, CONSOLE = True):
    log_file.write("%s\n"%string)
    if (CONSOLE==True):
        print(string)

if (os.path.exists(DATA_DIRECTORY)==False):
    log_print('DATA_DIRECTORY NOT EXISTED! CREATING ONE....')
    os.mkdir(DATA_DIRECTORY)
    log_print('DATA_DIRECTORY CREATED AT "%s"! '%DATA_DIRECTORY)
else:
    log_print('DATA_DIRECTORY EXISTED AT "%s"! '%DATA_DIRECTORY)


working directory: d:\HCMUS\Data Engineer\project
data directory: d:\HCMUS\Data Engineer\project\data
DATA_DIRECTORY EXISTED AT "d:\HCMUS\Data Engineer\project\data"! 


In [4]:


browser = webdriver.Chrome(executable_path=os.path.join(CURRENT_WORKING_DIRECTORY,"chromedriver.exe"))

c:\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [7]:

browser.get(r"https://www.iqair.com/vi/vietnam/ho-chi-minh-city")
WebDriverWait(browser, 3).until(
            EC.presence_of_element_located((By.CLASS_NAME,r"station-list__item")))
loc_table=browser.find_element(By.CLASS_NAME,'station-list')




locs=loc_table.find_elements_by_tag_name("a")
print("Table locations detected")

f=open(DATA_DIRECTORY+"locations_link.csv","w")
CRAWLING_SITE_URLS={}
writer=csv.DictWriter(f,fieldnames=['location','url',"vn_no_accent"])
writer.writeheader()
for loc in locs:
    loc_name=loc.text.strip()
    loc_url=loc.get_attribute("href").strip()
    no_accent=unidecode(loc_name).strip()
    writer.writerow({'location':loc_name,'url':loc_url,"vn_no_accent":no_accent})
    CRAWLING_SITE_URLS[no_accent]=loc_url
    log_print("Found location '%s' at the url: '%s'"%(loc_name,loc_url))
f.close()
pprint(CRAWLING_SITE_URLS)

Table locations detected
Found location 'Long Phuoc, District 94' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/long-phuoc-district-9'
Found location 'THCS Thanh My Loi37' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/thcs-thanh-my-loi'
Found location 'Vo Thi Sau High School37' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/vo-thi-sau-high-school'
Found location 'Mam non Hai Au Bay45' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/mam-non-hai-au-bay'
Found location 'IQAir Showroom - Saigon Pearl - Outdoor Air Quality Monitoring Station49' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/iqair-showroom-saigon-pearl-outdoor-air-quality-monitoring-station'
Found location 'VSTARSCHOOL57' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/vstarschool'
Found location 'Ap Xuan Thoi Dong 259' at the url: 'https://www.iqair.com/vi/vietnam/ho-chi-minh-city/ap-xuan-thoi-dong-2'
Found location 'Saigon South

In [ ]:

def find_button_by_text(text,strictly_matched=False):
    global browser
    all_buttons=browser.find_elements(By.TAG_NAME,"button")
    for bt in all_buttons:
        if (strictly_matched==False):
            if (text.lower().strip() in bt.text.strip().lower()):
                return bt
        else:
            if (text == bt.text.strip()):
                return bt
    return None




for SOURCE_NAME, url in CRAWLING_SITE_URLS.items():

    log_print("LOADING PAGE '%s' at: %s"%(SOURCE_NAME,url))
    browser.get(url)
    browser_actions=ActionChains(browser)
    try:
        WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH,r"/html/body/app-root/app-site-cookie-dialog/div/div/div[2]/button")))
        find_button_by_text("TÔI ĐỒNG Ý").click()
        log_print("Clicked button agree!")
    except:
        log_print("Unable to click the button agree, or not found button!")





    BRAND_DATA_DIRECTORY = os.path.join( DATA_DIRECTORY , SOURCE_NAME)
    if (os.path.exists(BRAND_DATA_DIRECTORY)==False):
        os.mkdir(BRAND_DATA_DIRECTORY)
        log_print("Folder '%s' was created"%BRAND_DATA_DIRECTORY)
    else:
        log_print("Folder '%s' was found!"%BRAND_DATA_DIRECTORY)


    for mode1 in ['daily','hourly']:
        for mode2 in ["AQI","PM2.5"]:
            try:
                WebDriverWait(browser, 3).until(EC.presence_of_element_located((By.XPATH,r"/html/body/app-root/app-site-cookie-dialog/div/div/div[2]/button")))
            except:
                pass

            if (mode1 == 'daily'):
                find_button_by_text('HẰNG NGÀY').click()
            else:
                find_button_by_text('HẰNG GIỜ').click()

            if (mode2=='AQI'):
                find_button_by_text('AQI').click()
            else:
                find_button_by_text('PM2.5').click()

            data_file_path=os.path.join(BRAND_DATA_DIRECTORY,"%s%s.csv"%(mode2.lower(),mode1))
            data=[]
            if (os.path.exists(data_file_path)==True):
                f=open(data_file_path,'r',encoding='utf8')
                for rec in csv.DictReader(f):
                    data.append(dict(rec))
                f.close()

            chart=browser.find_element(By.CLASS_NAME,'highcharts-series-group')
            bars=chart.find_elements(By.TAG_NAME,"rect")
            for b in bars:
                browser_actions.move_to_element(b).perform() 
                time.sleep(0.25)
                all_texts=browser.find_elements(By.TAG_NAME,"text")
                for t in all_texts:
                    try:
                        tspans=t.find_elements(By.TAG_NAME,"tspan")
                        if (len(tspans)==3):
                            print(t.text)
                            timestamp=tspans[0].text.strip()
                            rating = tspans[1].text.strip()
                            value = tspans[2].text.strip()
                            new_rec={"timestamp":timestamp,"value":value,"rating":rating}
                            new_data=True
                            for ind in range(len(data)):
                                if (data[ind]['timestamp']==timestamp):
                                    data[ind]=new_rec
                                    new_data=False
                                    break
                            if (new_data):
                                data.append(new_rec)
                    except:
                        pass
            pprint(data)
            f=open(data_file_path,"w",encoding='utf8')
            writer=csv.DictWriter(f,fieldnames=["timestamp","value",'rating'])
            writer.writeheader()
            for rec in data:
                writer.writerow(rec)
            f.close()
